# Exemple de résolution du TP 1

## Problème

2. Si l’on considère un groupe de 4 personnes, combien y a-t-il de façon de les répartir
dans 2 équipes de 2 ?

### Produire une solution de 3 manières différentes :

1. De manière analytique (en utilisant des formules mathématiques).

1. En utilisant une méthode de force brute (en énumérant toutes les combinaisons possibles de manière séquentielle).

1. En utilisant une méthode probabiliste (en simulant un grand nombre de combinaisons de 4 lancers de dé de manière aléatoire).

### Hint

La bibliothèque standard de Python inclus un module très utile pour le dénombrement : [`itertools`](https://docs.python.org/fr/3/library/itertools.html)

En particulier les itérateurs combinatoires :

* `product()` : produit cartésien, équivalent à une boucle for imbriquée
* `permutations()`
* `combinations()`
* `combinations_with_replacement()`

## Code
### Import et configuration

In [3]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.special import comb, factorial as fac

In [2]:
%config InlineBackend.figure_format="retina"  # For high DPI display

sns.set_style("darkgrid")
sns.set_context("notebook")

### Helper functions

In [3]:
def print_pct(p, prec=1):
    print(f"{p:.{prec}%}")

### Solution 1 (analytique)

$$\frac{\displaystyle\binom{N}{n}}{2!}$$

In [1]:
N = 4
n = 2

comb(N, n) / fac(2)

NameError: name 'comb' is not defined

### Solution 2 (force brute)

#### Définition des dès

In [5]:
PEOPLE = list("ABCD")
PEOPLE

['A', 'B', 'C', 'D']

In [6]:
people_df = pd.DataFrame(itertools.permutations(PEOPLE, 4))
people_df.columns = people_df.columns + 1
# df.columns = [f"p{i}" for i in df.columns + 1]
people_df

,1,2,3,4
0,A,B,C,D
1,A,B,D,C
2,A,C,B,D
3,A,C,D,B
4,A,D,B,C
5,A,D,C,B
6,B,A,C,D
7,B,A,D,C
8,B,C,A,D
9,B,C,D,A


In [7]:
people_df["g1"] = people_df.loc[:, 1:2].apply(frozenset, axis=1)
people_df["g2"] = people_df.loc[:, 3:4].apply(frozenset, axis=1)
people_df.drop(columns=range(1, 5), inplace=True)
people_df

,g1,g2
0,"(A, B)","(D, C)"
1,"(A, B)","(C, D)"
2,"(A, C)","(D, B)"
3,"(A, C)","(D, B)"
4,"(A, D)","(C, B)"
5,"(A, D)","(C, B)"
6,"(A, B)","(D, C)"
7,"(A, B)","(C, D)"
8,"(C, B)","(A, D)"
9,"(C, B)","(A, D)"


In [8]:
people_df.drop_duplicates(inplace=True)
people_df

,g1,g2
0,"(A, B)","(D, C)"
2,"(A, C)","(D, B)"
4,"(A, D)","(C, B)"
8,"(C, B)","(A, D)"
10,"(D, B)","(A, C)"
16,"(D, C)","(A, B)"


In [9]:
groups = people_df.apply(frozenset, axis=1)
groups

0     ((A, B), (D, C))
2     ((A, C), (D, B))
4     ((C, B), (A, D))
8     ((A, D), (C, B))
10    ((A, C), (D, B))
16    ((D, C), (A, B))
dtype: object

In [10]:
groups.drop_duplicates(inplace=True)
groups

0    ((A, B), (D, C))
2    ((A, C), (D, B))
4    ((C, B), (A, D))
dtype: object

In [11]:
def make_equal_groups(df: pd.DataFrame, group_n: int):
    df.columns = df.columns + 1
    people_n = len(df.columns)
    df["g1"] = df.loc[:, 1:group_n].apply(frozenset, axis=1)
    df["g2"] = df.loc[:, group_n + 1:people_n].apply(frozenset, axis=1)
    df.drop(columns=range(1, people_n + 1), inplace=True)
    df.drop_duplicates(inplace=True)
    groups = df.apply(frozenset, axis=1)
    groups.drop_duplicates(inplace=True)
    return groups.reset_index(drop=True)

In [12]:
ppl_df = pd.DataFrame(itertools.permutations(PEOPLE, 4))

make_equal_groups(ppl_df, group_n=2)

0    ((A, B), (D, C))
1    ((A, C), (D, B))
2    ((C, B), (A, D))
dtype: object

---

### Solution 3 (probabiliste)

La bibliothèque open source NumPy est destinée à manipuler des matrices ou tableaux multidimensionnels.
Elle propose également de nombreuses fonctions mathématiques et notemment des outils pour générer et manipuler des variables aléatoires.

Les versions récentes de NumPy favorise l'utilisation du générateur aléatoire par défaut comme expliqué dans le lien ci-dessous :

https://numpy.org/doc/stable/reference/random/generator.html

TLDR : 
```python
rng = np.random.default_rng(seed)
rng.integers()
# ou
rng.random()
# ou
rng.permutation()
# ...
```

In [13]:
SEED = 1234  # Pour la reproductibilité des résultats (valeur arbitraire)

RNG = np.random.default_rng(SEED)

In [14]:
def rdm_permutation(rng, people):
    return rng.permutation(people).tolist()

In [16]:
rdm_permutation(RNG, PEOPLE)

['B', 'C', 'D', 'A']

In [17]:
def rdm_perm_sim(rng, people, perm_n):
    return [rdm_permutation(rng, people) for _ in range(perm_n)]

In [18]:
SIM_N = 100
groups_sim_df = pd.DataFrame(rdm_perm_sim(RNG, PEOPLE, SIM_N))
groups_sim_df

,0,1,2,3
0,B,D,A,C
1,A,B,D,C
2,B,D,A,C
3,B,A,C,D
4,A,B,C,D
...,...,...,...,...
95,D,C,A,B
96,B,A,D,C
97,A,C,B,D
98,D,B,A,C


In [19]:
make_equal_groups(groups_sim_df, group_n=2)

0    ((A, C), (D, B))
1    ((A, B), (C, D))
2    ((C, B), (A, D))
dtype: object

### (Optionnel) Visualisation de la distribution

Distribution des sommes des lancers des 4 dés

In [53]:
# sns.histplot(rolls_sim_df, x="sums", discrete=True, stat="probability")
# plt.show()

TD 1 Analytique

In [ ]:
N = 50
n= 3
somme = 1

for i in range (n) :
    /* question a */somme = somme *(N-i)
/* question b */ somme/=fac(n)    
print(somme)       

19600.0


$$\frac{\displaystyle\binom{N}{n}}{2!}$$

Solution 1 (analytique) de la 6) a

In [5]:
import math

Nombre_de_cartes = 52
cartes_distribués = 2
nombre_as = 4
nombre_val10 = 16
combinaisons = math.comb(Nombre_de_cartes, cartes_distribués)
combi_de21 = nombre_as * nombre_val10
print(combi_de21 / combinaisons)



0.048265460030165915


---

Solution 1 (analytique) de la 6) b

In [1]:
#Valeur = 15 on cherche la probabilité que la prochaine carte fait perdre le joueur
cartes_restantes=50
nmbr_prochaines_cartesperd=27
tirage=1
chance_perte=27/50
print(chance_perte)

0.54


Solution 3 manière probabiliste 6.a)